In [10]:
import requests
from datetime import datetime, timedelta

# Function to format date string
def format_date(date):
    return date.strftime("%Y-%m-%d")

# Define the URL and headers
url = "https://www.sharesansar.com/ajaxtodayshareprice"
headers = {
    "accept": "*/*",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    "cookie": "_gid=GA1.2.317488179.1716993804; _ga_FPXFBP3MT9=GS1.1.1716997443.2.1.1717000661.60.0.0; _ga=GA1.2.1019898196.1716993802; XSRF-TOKEN=eyJpdiI6InhJVzg1NUcvYUhOY25HaThEQ0tSSGc9PSIsInZhbHVlIjoidzJIZVBpdExabEhqMHFZOGxQMTdXUHVpU0R6eklCVDFsZnAvOTVxaS94b0lDd0VhWHA3aGFYd1RleHhiRVNKVjEwZXZlaTF6RDB5aVJaSnE1WUU1eXNycFg3VVp3eDM0R0lNUnRwc2U1ZklBTU92QWk0bW9RRngyWTB5WmJyL2UiLCJtYWMiOiIxYjhhYTVkMTE1YzNjNzk4NDQ4NGJkMTU2Nzk5YWU1MGJlNGE5MTEzOWZhZDgxOGM1ODI3MTdkMjRlMjNmYmY0In0%3D; sharesansar_session=eyJpdiI6IjJBUmd0NHErMzBKcE9YMVhXUlYxTnc9PSIsInZhbHVlIjoiTytKeTV3dnZ2Tko0Q1ZLcDFNcGFJNEs1MXA3WUpXNUlibE5HaFhnUXR4OC9TdjQ5MGpwckpGb2ZMR0dCUlR6cVV1ZElUMUE4Q2taNnZDbWR5YlRrQk5rYWowSkI3VUJSZlQ5K1dkYnFRV3ZmYlhQMGVySC9pWjFPUzJ3Y2VIbEsiLCJtYWMiOiJmZDE1MjU5MDYzZDI5MjFjOTc0MDEwODM1NzhiMGU0NzI1Zjg2M2Y4ZWRkOTgzYjQ5NmVkM2FiZmYwMzgzOTk5In0%3D",
    "dnt": "1",
    "origin": "https://www.sharesansar.com",
    "priority": "u=1, i",
    "referer": "https://www.sharesansar.com/today-share-price",
    "sec-ch-ua": '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    "sec-ch-ua-mobile": "?1",
    "sec-ch-ua-platform": '"Android"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Mobile Safari/537.36",
    "x-requested-with": "XMLHttpRequest"
}

# Start and end dates
start_date = datetime.strptime("2024-05-23", "%Y-%m-%d")
end_date = datetime.strptime("2024-05-26", "%Y-%m-%d")

# Loop through dates and make requests
current_date = start_date
while current_date <= end_date:
    data = {
        "_token": "4PHlXikMPQxZRmBz3Ded77w7FzmQCrovhiS4Sv00",
        "sector": "all_sec",
        "date": format_date(current_date)
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        response_html = response.text
        print("Response for", data['date'], ":")
        print(response_html)
    else:
        print("Request failed for", format_date(current_date), "with status code:", response.status_code)
    
    # Move to the next date
    current_date += timedelta(days=1)


Request failed for 2024-05-23 with status code: 419
Request failed for 2024-05-24 with status code: 419
Request failed for 2024-05-25 with status code: 419
Request failed for 2024-05-26 with status code: 419


In [11]:
import re
import csv

# Define regex pattern to match the date
date_pattern = r'<h5>As of : <span class="text-org">(\d{4}-\d{2}-\d{2})</span></h5>'
date_matches = re.findall(date_pattern, response_html, re.DOTALL)

# Define regex pattern to match each table section
table_pattern = r'<h5>As of : <span class="text-org">(\d{4}-\d{2}-\d{2})</span></h5>(.*?)</table>'
table_matches = re.findall(table_pattern, response_html, re.DOTALL)

# Extract <thead> section to get headers
thead_pattern = r'<thead>(.*?)</thead>'
thead_matches = re.findall(thead_pattern, response_html, re.DOTALL)

# Extract headers
headers = []
if thead_matches:
    headers = re.findall(r'<th.*?>(.*?)</th>', thead_matches[0], re.DOTALL)
    headers = [re.sub(r'<.*?>', '', header).strip() for header in headers]

headers = ['Date'] + headers
print(headers)


['Date', 'S.No', 'Symbol', 'Conf.', 'Open', 'High', 'Low', 'Close', 'VWAP', 'Vol', 'Prev. Close', 'Turnover', 'Trans.', 'Diff', 'Range', 'Diff %', 'Range %', 'VWAP %', '52 Weeks High', '52 Weeks Low']


In [12]:
# Extract <tbody> section to get data rows
tbody_pattern = r'<tbody>(.*?)</tbody>'
row_pattern = r'<tr>(.*?)</tr>'
cell_pattern = r'<t[dh].*?>(.*?)</t[dh]>'

data = []

for date, table_content in table_matches:
    # Check if the table has valid data
    tbody_matches = re.findall(tbody_pattern, table_content, re.DOTALL)
    if not tbody_matches or 'No Record Found.' in tbody_matches[0]:
        continue
    
    for tbody in tbody_matches:
        row_matches = re.findall(row_pattern, tbody, re.DOTALL)
        for row in row_matches:
            cells = re.findall(cell_pattern, row, re.DOTALL)
            # Clean up cell values
            cells = [re.sub(r'<.*?>', '', cell).strip() for cell in cells]
            data.append([date] + cells)

In [13]:
# Write to CSV
csv_filename = "23_26_nepse_stock_data.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerows(data)

print(f"Data has been written to {csv_filename}")

Data has been written to 23_26_nepse_stock_data.csv
